In [1]:
import qiskit
qiskit.__version__

'1.3.2'

In [2]:
import numpy as np
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.primitives import StatevectorSampler
import matplotlib.pyplot as plt
from qiskit.visualization import plot_histogram

In [3]:
#step 1 : generate alice's key and basis

n = 16 # number of qubits
alice_key = np.random.randint(0, high=2**n) # generates a random number between 0 and 2^n(here 2^16)
alice_key = np.binary_repr(alice_key, n) # converts the random number to binary exactly 16 digits long

In [4]:
#step 2: encode alice's key according to the basis into the circuit, if the key has 0 bit then use computational basis and if 
# if it has 1 use the diagonal basis

qr = QuantumRegister(n, name='qr')
cr = ClassicalRegister(n, name='cr')
bb92 = QuantumCircuit(qr, cr, name='bb92')

# since ibm has intializes all qubits as |0> as default we will apply x gate to change it to |1> is the key has 1
for index, digit in enumerate(alice_key):
    if digit == '1':
        bb92.x(qr[index])
        bb92.h(qr[index])

Now, since we don't have another quantum computer for simplicity, we will assume that bob measures at states at the end with his own random basis without knowing the previous states or qubits or alice's keys and basis. 

In [5]:
#step 3: generate bob's basis
bob_basis = np.random.randint(2, size=n) # generates n size array of 0 or 1 (basically computational and diagonal basis resp.)

In [6]:
#step 4: bob will measure the circuit at the end with his own basis states
# we apply the hadamard gate for diagonal basis if the basis indicates (i.e 1 )
for index in range(len(qr)):       
    if bob_basis[index] == 1:   
        bb92.h(qr[index])

#measure all qubits
bb92.measure_all()

In [7]:
sampler = StatevectorSampler()

job = sampler.run([bb92],shots=1)
result = job.result()
counts = result[0].data.meas.get_counts()

In [8]:
# Result of the measure is Bob's key candidate
bob_key = list(counts)[0]
bob_key = bob_key[::-1]

print("alice_key: ", alice_key)
#print("alice_basis: ", alice_basis)
print("bob_key: ",bob_key)
print("bob_basis: ", bob_basis)

alice_key:  0110010111100101
bob_key:  0010010001101101
bob_basis:  [0 0 1 0 0 1 0 0 0 0 1 0 1 0 1 1]


In [9]:
#step 5: classical post-processing

keep = []
discard = []
#for qubit, basis in enumerate(zip(alice_basis, bob_basis)):
for qubit, basis in enumerate(zip(alice_key, bob_basis)):
    if np.int64(basis[0]) == basis[1]:
        print("Same choice for qubit: {}, basis: {}" .format(qubit, basis[0])) 
        keep.append(qubit)
    else:
        print("Different choice for qubit: {}, Alice has {}, Bob has {}" .format(qubit, basis[0], basis[1]))
        #print(type(np.int64(basis[0])))
        #print(type(basis[1]))
        discard.append(qubit)

Same choice for qubit: 0, basis: 0
Different choice for qubit: 1, Alice has 1, Bob has 0
Same choice for qubit: 2, basis: 1
Same choice for qubit: 3, basis: 0
Same choice for qubit: 4, basis: 0
Same choice for qubit: 5, basis: 1
Same choice for qubit: 6, basis: 0
Different choice for qubit: 7, Alice has 1, Bob has 0
Different choice for qubit: 8, Alice has 1, Bob has 0
Different choice for qubit: 9, Alice has 1, Bob has 0
Same choice for qubit: 10, basis: 1
Same choice for qubit: 11, basis: 0
Different choice for qubit: 12, Alice has 0, Bob has 1
Different choice for qubit: 13, Alice has 1, Bob has 0
Different choice for qubit: 14, Alice has 0, Bob has 1
Same choice for qubit: 15, basis: 1


In [10]:
keep

[0, 2, 3, 4, 5, 6, 10, 11, 15]

In [11]:
acc = 0
for bit in zip(alice_key, bob_key):
    if bit[0] == bit[1]:
        acc += 1

print('Percentage of qubits to be discarded according to table comparison: ', len(keep)/n)
print('Measurement convergence by additional chance: ', acc/n) 

Percentage of qubits to be discarded according to table comparison:  0.5625
Measurement convergence by additional chance:  0.75


In [14]:
new_alice_key = [alice_key[qubit] for qubit in keep]
new_bob_key = [bob_key[qubit] for qubit in keep]

acc = 0
for bit in zip(new_alice_key, new_bob_key):
    if bit[0] == bit[1]:
        acc += 1        

print(new_alice_key)
print(new_bob_key)
print('Percentage of similarity between the keys: ', acc/len(new_alice_key))              

['0', '1', '0', '0', '1', '0', '1', '0', '1']
['0', '1', '0', '0', '1', '0', '1', '0', '1']
Percentage of similarity between the keys:  1.0


# Now for the case when there is an EVE eavesdropping

In [15]:
#step 1 : generate alice's key and basis

n = 16 # number of qubits
alice_key = np.random.randint(0, high=2**n) # generates a random number between 0 and 2^n(here 2^16)
alice_key = np.binary_repr(alice_key, n) # converts the random number to binary exactly 16 digits long


#step 2: encode alice's key according to the basis into the circuit

qr = QuantumRegister(n, name='qr')
cr = ClassicalRegister(n, name='cr')
evebb92 = QuantumCircuit(qr, cr, name='evebb92')

# since ibm has intializes all qubits as |0> as default we will apply x gate to change it to |1> is the key has 1
for index, digit in enumerate(alice_key):
    if digit == '1':
        evebb92.x(qr[index])
        evebb92.h(qr[index])


#step 3: generate eve's basis
eve_basis = np.random.randint(2, size=n) # generates n size array of 0 or 1 (basically computational and diagonal basis resp.)

#step 4: bob will measure the circuit at the end with his own basis states
# we apply the hadamard gate for diagonal basis if the basis indicates (i.e 1 )
for index in range(len(qr)):       
    if bob_basis[index] == 1:   
        evebb92.h(qr[index])

#measure all qubits
evebb92.measure_all()

sampler = StatevectorSampler()

job = sampler.run([evebb92],shots=1)
result = job.result()
counts = result[0].data.meas.get_counts()

# Result of the measure is eve's key candidate
eve_key = list(counts)[0]
eve_key = eve_key[::-1]

up untill now eve has been playing the role of bob to fool alice, now she has to send these qubits to bob, acting like alice.Eve has measured the state causing qubits to collapse in different eigenstates. This property is not easy to implement in qiskit because measurement results are stored in classical registers, while the qubits themselves are "unchanged". Therefore we need to update Eve's qubits to the new altered states starting from the results of the measures (Eve's key), reversing the instructions that Eve has executed.

In [16]:
qr = QuantumRegister(n, name='qr')
cr = ClassicalRegister(n, name='cr')
bobbb92 = QuantumCircuit(qr, cr, name='bobbb92')

for qubit, basis in enumerate(zip(alice_key, eve_key)):
    if basis[0] == basis[1]:
        print("Same choice for qubit: {}, basis: {}" .format(qubit, basis[0]))
    else:
        print("Different choice for qubit: {}, Alice has {}, Eve has {}" .format(qubit, basis[0], basis[1]))
        if eve_key[qubit] == alice_key[qubit]:
            bobbb92.h(qr[qubit])
        else:
            if basis[0] == 0 and basis[1] == 1:
                bobbb92.h(qr[qubit])
                bobbb92.x(qr[qubit])
            else:
                bobbb92.x(qr[qubit])
                bobbb92.h(qr[qubit])

Different choice for qubit: 0, Alice has 1, Eve has 0
Same choice for qubit: 1, basis: 0
Same choice for qubit: 2, basis: 1
Same choice for qubit: 3, basis: 1
Different choice for qubit: 4, Alice has 1, Eve has 0
Same choice for qubit: 5, basis: 0
Same choice for qubit: 6, basis: 1
Same choice for qubit: 7, basis: 0
Different choice for qubit: 8, Alice has 1, Eve has 0
Same choice for qubit: 9, basis: 0
Same choice for qubit: 10, basis: 1
Same choice for qubit: 11, basis: 0
Same choice for qubit: 12, basis: 0
Same choice for qubit: 13, basis: 1
Different choice for qubit: 14, Alice has 0, Eve has 1
Same choice for qubit: 15, basis: 1


In [18]:
bob_basis = np.random.randint(2, size=n) 

for index in range(len(qr)):       
    if bob_basis[index] == 1:   
        bobbb92.h(qr[index])

bobbb92.measure_all()

sampler = StatevectorSampler()

job = sampler.run([bobbb92],shots=1)
result = job.result()
counts = result[0].data.meas.get_counts()

bob_key = list(counts)[0]
bob_key = bob_key[::-1]

In [20]:
keep = []
discard = []
for qubit, basis in enumerate(zip(alice_key, bob_basis)):
    if np.int64(basis[0]) == basis[1]:
        print("Same choice for qubit: {}, basis: {}" .format(qubit, basis[0])) 
        keep.append(qubit)
    else:
        print("Different choice for qubit: {}, Alice has {}, Bob has {}" .format(qubit, basis[0], basis[1]))
        discard.append(qubit)

acc = 0
for bit in zip(alice_key, bob_key):
    if bit[0] == bit[1]:
        acc += 1

print('Percentage of qubits to be discarded according to table comparison: ', len(keep)/n)
print('Measurement convergence by additional chance: ', acc/n) 

new_alice_key = [alice_key[qubit] for qubit in keep]
new_bob_key = [bob_key[qubit] for qubit in keep]

acc = 0
for bit in zip(new_alice_key, new_bob_key):
    if bit[0] == bit[1]:
        acc += 1        

print(new_alice_key)
print(new_bob_key)
print('Percentage of similarity between the keys: ', acc/len(new_alice_key))

Same choice for qubit: 0, basis: 1
Different choice for qubit: 1, Alice has 0, Bob has 1
Same choice for qubit: 2, basis: 1
Same choice for qubit: 3, basis: 1
Different choice for qubit: 4, Alice has 1, Bob has 0
Same choice for qubit: 5, basis: 0
Same choice for qubit: 6, basis: 1
Different choice for qubit: 7, Alice has 0, Bob has 1
Same choice for qubit: 8, basis: 1
Same choice for qubit: 9, basis: 0
Different choice for qubit: 10, Alice has 1, Bob has 0
Same choice for qubit: 11, basis: 0
Same choice for qubit: 12, basis: 0
Same choice for qubit: 13, basis: 1
Same choice for qubit: 14, basis: 0
Different choice for qubit: 15, Alice has 1, Bob has 0
Percentage of qubits to be discarded according to table comparison:  0.6875
Measurement convergence by additional chance:  0.625
['1', '1', '1', '0', '1', '1', '0', '0', '0', '1', '0']
['1', '1', '0', '0', '0', '1', '0', '0', '0', '1', '0']
Percentage of similarity between the keys:  0.8181818181818182


In [21]:
if (acc//len(new_alice_key) == 1):
    print("\nKey exchange has been successfull")
    print("New Alice's key: ", new_alice_key)
    print("New Bob's key: ", new_bob_key)
else:
    print("\nKey exchange has been tampered! Check for eavesdropper or try again")
    print("New Alice's key is invalid: ", new_alice_key)
    print("New Bob's key is invalid: ", new_bob_key)


Key exchange has been tampered! Check for eavesdropper or try again
New Alice's key is invalid:  ['1', '1', '1', '0', '1', '1', '0', '0', '0', '1', '0']
New Bob's key is invalid:  ['1', '1', '0', '0', '0', '1', '0', '0', '0', '1', '0']


### References: 

-https://github.com/qiskit-community/qiskit-community-tutorials/blob/master/awards/teach_me_qiskit_2018/cryptography/Cryptography.ipynb